In [119]:
import openai
from openai import AzureOpenAI, AsyncAzureOpenAI
from tqdm.notebook import tqdm
import os
from dotenv import load_dotenv, find_dotenv
import requests
import base64

GPT_3_5_TURBO = "gpt-3.5-turbo"
GPT_4_TURBO_PREVIEW = "gpt-4-turbo-preview"
GPT_4 = 'gpt-4'
GPT_4o = 'gpt-4o'

def get_openai_api_key():
    _ = load_dotenv(find_dotenv())

    return os.getenv("OPENAI_API_KEY")

OPENAI_API_KEY = get_openai_api_key()
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [156]:
import json


EVAL_FOLDER = '../../data/russian-english/cards/eval_results/ru_eng_'
def get_eval_results_from_file(file_name):
    data = []
    file_path = EVAL_FOLDER + file_name
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data


RU_ENG_OUTPUT_FOLDER = '../../data/russian-english/cards/test_cards/ru_eng_'
def get_ru_eng_cards_from_file(file_name):
    data = []
    file_path = OUTPUT_FOLDER + file_name
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data


RU_FINN_FOLDER_WITH_JSON = '../../data/russian-finnish/cards/curated_platform_cards/'
def get_ru_finn_cards_from_file(file_name):
    data = []
    file_path = RU_FINN_FOLDER_WITH_JSON + file_name
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    res = []
    for card in data:
        if 'isMarkedDeleted' not in card.keys():
            res.append(card)
    return res


OUTPUT_FOLDER = '../../data/russian-english/cards/test_cards/ru_eng_'
def write_cards_to_file(file_name, cards):
    file_path = OUTPUT_FOLDER + file_name
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(cards, f, ensure_ascii=False, indent=2)
        
        
EVAL_FOLDER = '../../data/russian-english/cards/eval_results/ru_eng_'
def write_eval_results_to_file(file_name, results):
    file_path = EVAL_FOLDER + file_name
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

In [144]:
def generate_phrase_cards_and_evaluate(ru_finn_cards, ru_eng_cards, ru_eng_evals, model=GPT_4o):
    cards = []
    eval_results = []
    for ru_finn, ru_eng, eval_result in tqdm(list(zip(ru_finn_cards, ru_eng_cards, ru_eng_evals))):
        try:
            ru_word, ru_sentence, finn_word, finn_sentence, card_id = ru_finn.values()
            if ru_sentence != '':
                cards.append(ru_eng)
                eval_results.append(eval_result)
                continue
                
            system_prompt = '''You are a multilingual assistant who is proficient in Russian, Finnish and English.'''

            user_prompt = f"""
            Translate the given Russian word or phrase: '{ru_word}' into clear and natural English, reflecting its meaning and context as the primary focus. Use the Finnish equivalent: '{finn_word}' as a supportive reference to help clarify or refine the exact context if needed. The goal is to create an English translation that is accurate, fluent, and authentic to native speakers, avoiding overly complex or literal phrasing.

            Please provide the English translation of the word or phrase and nothing else.
            """

            response = client.chat.completions.create(
              model=model,
              messages=[
                {"role": "system", "content": system_prompt.strip()},
                {"role": "user", "content": user_prompt.strip()}
              ]
            )
            
            tr_word = response.choices[0].message.content.strip()

            card = {}
            card["wordFirstLang"] = ru_word
            card["sentenceFirstLang"] = ""
            card["wordSecondLang"] = tr_word
            card["sentenceSecondLang"] = ""
            card["id"] = card_id
            cards.append(card)
            
            user_prompt = f"""
            As an AI model, your task is to evaluate the correctness and naturalness of English translations for given Russian and Finnish words or phrases. Check if the English translation accurately conveys the meaning and context of the Russian and Finnish versions, and whether it sounds natural to a native speaker. Your evaluation does not need to suggest the best possible translation, only confirm that it is good enough and identify any issues if present.

            When suggesting corrections, provide only the final corrected English translation. If no correction is needed, set `suggestedFix` to `null`.

            Here are the words or phrases:  
            - Word or Phrase in Russian: {ru_word}  
            - Word or Phrase in Finnish: {finn_word}  
            - Word or Phrase in English: {tr_word}  

            Respond in JSON format using the following structure:  
            {{
              "translationAccuracy": {{
                "isCorrect": true/false,
                "explanation": "Detailed explanation if there is an issue or why it's correct",
                "suggestedFix": "Suggested correction if there is an issue or null if no correction is needed"
              }}
            }}
            """
            
            response = client.chat.completions.create(
              model="gpt-4o",
              response_format={ "type": "json_object" },
              messages=[
                {"role": "system", "content": system_prompt.strip()},
                {"role": "user", "content": user_prompt.strip()}
              ]
            )

            res = json.loads(response.choices[0].message.content.strip())
            res['id'] = card['id']
            eval_results.append(res)
            
        except Exception as e:
            print(ru_finn['id'])
    return cards, eval_results

In [145]:
def improve_cards_with_eval(ru_finn_cards, ru_eng_cards, ru_eng_evals, model=GPT_4o):
    cards = []
    eval_results = []
    for ru_finn, ru_eng, eval_result in tqdm(list(zip(ru_finn_cards, ru_eng_cards, ru_eng_evals))):
        try:
            ru_word, ru_sentence, finn_word, finn_sentence, card_id = ru_finn.values()
            _, _, eng_word, eng_sentence, _ = ru_eng.values()
            if ru_sentence != '':
                cards.append(ru_eng)
                eval_results.append(eval_result)
                continue
            
            is_correct, explanation, suggested_fix = eval_result['translationAccuracy'].values()

            if is_correct:
                cards.append(ru_eng)
                eval_results.append(eval_result)
                continue
                
            system_prompt = '''You are a multilingual assistant who is proficient in Russian, Finnish and English.'''

            user_prompt = f"""
            Translate the given Russian word or phrase: '{ru_word}' into clear and natural English, reflecting its meaning and context as the primary focus. Use the Finnish equivalent: '{finn_word}' as a supportive reference to help clarify or refine the exact context if needed. The goal is to create an English translation that is accurate, fluent, and authentic to native speakers, avoiding overly complex or literal phrasing.

            Consider the following:  

            - Existing English translation: '{eng_word}'  
            - Issues identified: '{explanation}'  
            - Suggested improvement: '{suggested_fix}'  

            Using this information, provide a natural English translation that prioritizes the Russian meaning while considering the Finnish phrase for additional context. Respond with **only the final English translation** without any additional explanations.
            """

            response = client.chat.completions.create(
              model=model,
              messages=[
                {"role": "system", "content": system_prompt.strip()},
                {"role": "user", "content": user_prompt.strip()}
              ]
            )
            
            tr_word = response.choices[0].message.content.strip()

            card = {}
            card["wordFirstLang"] = ru_word
            card["sentenceFirstLang"] = ""
            card["wordSecondLang"] = tr_word
            card["sentenceSecondLang"] = ""
            card["id"] = card_id
            cards.append(card)
            
            user_prompt = f"""
            As an AI model, your task is to evaluate the correctness and naturalness of English translations for given Russian and Finnish words or phrases. Check if the English translation accurately conveys the meaning and context of the Russian and Finnish versions, and whether it sounds natural to a native speaker. Your evaluation does not need to suggest the best possible translation, only confirm that it is good enough and identify any issues if present.

            When suggesting corrections, provide only the final corrected English translation. If no correction is needed, set `suggestedFix` to `null`.

            Here are the words or phrases:  
            - Word or Phrase in Russian: {ru_word}  
            - Word or Phrase in Finnish: {finn_word}  
            - Word or Phrase in English: {tr_word}  

            Respond in JSON format using the following structure:  
            {{
              "translationAccuracy": {{
                "isCorrect": true/false,
                "explanation": "Detailed explanation if there is an issue or why it's correct",
                "suggestedFix": "Suggested correction if there is an issue or null if no correction is needed"
              }}
            }}
            """
            
            response = client.chat.completions.create(
              model="gpt-4o",
              response_format={ "type": "json_object" },
              messages=[
                {"role": "system", "content": system_prompt.strip()},
                {"role": "user", "content": user_prompt.strip()}
              ]
            )

            res = json.loads(response.choices[0].message.content.strip())
            res['id'] = card['id']
            eval_results.append(res)
            
        except Exception as e:
            print(ru_finn['id'])
    return cards, eval_results

In [146]:
file_names = [f'sm1_new_kap{i}.json' for i in range(1, 9)]

In [147]:
file_names[2]

'sm1_new_kap3.json'

In [152]:
tmp_eval = get_eval_results_from_file(file_names[5])
tmp_ru_eng = get_ru_eng_cards_from_file(file_names[5])
tmp_ru_finn = get_ru_finn_cards_from_file(file_names[5])

lol
lol


In [ ]:
tmp_cards, tmp_res = generate_phrase_cards_and_evaluate(tmp_ru_finn, tmp_ru_eng, tmp_eval)

  0%|          | 0/233 [00:00<?, ?it/s]

2249
2250


In [150]:
len(tmp_eval), len(tmp_ru_finn)

(233, 239)

In [138]:
improved_cards, improved_evals = improve_cards_with_eval(tmp_ru_finn, tmp_cards, tmp_res)

  0%|          | 0/231 [00:00<?, ?it/s]

2249
2250


In [139]:
improved_cards_v2, improved_evals_v2 = improve_cards_with_eval(tmp_ru_finn, improved_cards, improved_evals)

  0%|          | 0/229 [00:00<?, ?it/s]

2249
2250


In [140]:
sum([res['translationAccuracy']['isCorrect'] for res in improved_cards_v2 if len(res.items()) == 2]) / len([res['translationAccuracy']['isCorrect'] for res in improved_cards_v2 if len(res.items()) == 2])

ZeroDivisionError: division by zero

In [134]:
sum([res['translationAccuracy']['isCorrect'] for res in improved_evals if len(res.items()) == 2]) / len([res['translationAccuracy']['isCorrect'] for res in improved_evals if len(res.items()) == 2])

0.935064935064935

In [129]:
improved_evals

[{'translationAccuracy': {'isCorrect': False,
   'explanation': "The word 'гардероб' in Russian and 'vaatehuone' in Finnish both refer to a 'closet' or 'walk-in closet' rather than a 'wardrobe'. 'Wardrobe' typically refers to a piece of furniture for storing clothes, whereas 'гардероб' and 'vaatehuone' suggest a room or area designated for storing clothes.",
   'suggestedFix': 'Closet'},
  'id': 2155},
 {'translationAccuracy': {'isCorrect': True,
   'explanation': "The English translation 'Reserved sauna time' accurately conveys the meaning of 'время для сауны' in Russian and 'saunavuoro' in Finnish. 'Reserved sauna time' is natural and understandable to a native English speaker.",
   'suggestedFix': None},
  'id': 2178},
 {'translationAccuracy': {'isCorrect': False,
   'explanation': "The given English translation 'I see' generally conveys the meaning of understanding or realization, which aligns with 'Ага' in Russian and 'Ahaa' in Finnish in certain contexts. However, 'Ага' and 'Ahaa

In [130]:
sum([res['translationAccuracy']['isCorrect'] for res in tmp_res if len(res.items()) == 2]) / len([res['translationAccuracy']['isCorrect'] for res in tmp_res if len(res.items()) == 2])

0.9090909090909091

In [63]:
[res for res in tmp_res]

[{'translationAccuracy': {'isCorrect': True,
   'explanation': "The English translation 'What time is it?' accurately reflects the meaning of the Russian phrase 'Сколько времени?' and the Finnish phrase 'Mitä kello on?'. Both phrases are commonly used to ask for the current time in their respective languages.",
   'suggestedFix': None},
  'id': 1933},
 {'translationAccuracy': {'isCorrect': True,
   'explanation': "The English phrase 'at one o'clock' accurately captures the meaning of both the Russian 'в час' and the Finnish 'kello yksi (yhdeltä)', indicating the same time. The translation is correct and sounds natural.",
   'suggestedFix': None},
  'id': 1941},
 {'translationAccuracy': {'isCorrect': True,
   'explanation': "The English phrase 'At two o'clock' accurately reflects the meaning of both the Russian phrase 'в два часа' and the Finnish phrase 'kello kaksi (kahdelta)' with no loss of context or nuance.",
   'suggestedFix': None},
  'id': 1942},
 {'translationAccuracy': {'isCor

In [8]:
sum([res['translationAccuracy']['isCorrect'] for res in tmp_res if res['translationAccuracy']['isCorrect'] is not None]) / len([res['translationAccuracy']['isCorrect'] for res in tmp_res])

0.9625

In [199]:
def generate_complete_cards_and_evaluate(ru_finn_cards, model=GPT_4o):
    cards = []
    eval_results = []
    for ru_finn in tqdm(ru_finn_cards):
        try:
            ru_word, ru_sentence, finn_word, finn_sentence, card_id = ru_finn.values()
            if ru_sentence == '':
                #cards.append(ru_eng)
                #eval_results.append(eval_result)
                continue
                
            system_prompt = '''You are a multilingual assistant who is proficient in Russian, Finnish and English.'''

            user_prompt = f"""
            Translate the given Russian word or phrase along with its Finnish translation into English, and then translate the provided Russian sentence, incorporating the English translation of the word or phrase. Use synonyms or related terms where necessary to convey the intended meaning and maintain naturalness in English.

            Given word or phrase (Russian): '{ru_word}'
            Given word or phrase (Finnish): '{finn_word}'

            Given sentence (Russian): '{ru_sentence}'
            Given sentence (Finnish): '{finn_sentence}'

            Return the English translated word or phrase and the English translated sentence, separated by a semicolon (;).
            """

            response = client.chat.completions.create(
              model=model,
              messages=[
                {"role": "system", "content": system_prompt.strip()},
                {"role": "user", "content": user_prompt.strip()}
              ]
            )
            
            tr_word, tr_sentence = response.choices[0].message.content.strip().split(';')

            card = {}
            card["wordFirstLang"] = ru_word
            card["sentenceFirstLang"] = ru_sentence
            card["wordSecondLang"] = tr_word.strip()
            card["sentenceSecondLang"] = tr_sentence.strip()
            card["id"] = card_id
            cards.append(card)
            
            user_prompt = f"""
            Evaluate the correctness of an English word and sentence based on their translations from Russian and Finnish. You will receive a word in Russian, Finnish, and its translation in English, as well as a sentence in Russian, Finnish, and its translation in English. Your task is to assess the quality of the English sentence, the usage of the English word in the sentence, and the accuracy of the translations from Russian and Finnish to English. For each evaluation point, provide a detailed explanation of your judgment and suggest fixes where applicable, either to the English word, the English sentence, or both.

            Please ensure that the English sentence is grammatically correct and natural. Suggest a corrected version if necessary. Verify that the English sentence contains the English word in some form and suggest using synonyms or related terms if the word is missing. Prioritize naturalness and correctness. Ensure that the translations of both the word and sentence from Russian and Finnish to English are accurate and provide corrections if necessary.

            Respond in JSON format with the following structure:
            {{
              "englishSentence": {{
                "isCorrect": true/false,
                "explanation": "Detailed explanation if there is an issue or why it's correct.",
                "suggestedFix": "Suggested corrected sentence if there is an issue, or null if not applicable."
              }},
              "wordUsage": {{
                "isCorrect": true/false,
                "explanation": "Detailed explanation if there is an issue or why it's correct.",
                "suggestedFixSentence": "Suggested corrected sentence if the word usage is incorrect, or null if not applicable.",
                "suggestedFixWord": "Suggested corrected word if the word usage is incorrect, or null if not applicable."
              }},
              "translationAccuracy": {{
                "isCorrect": true/false,
                "explanation": "Detailed explanation if there is an issue or why it's correct.",
                "suggestedFix": "Suggested correction for translation issues, or null if not applicable."
              }}
            }}

            Here are the provided word and sentence in Russian, Finnish, and English:

            - Word in Russian: {ru_word}
            - Word in Finnish: {finn_word}
            - Word in English: {tr_word}
            - Sentence in Russian: {ru_sentence}
            - Sentence in Finnish: {finn_sentence}
            - Sentence in English: {tr_sentence}

            Please adhere to this structure to ensure clear, actionable feedback for each evaluation point.
            """
            
            response = client.chat.completions.create(
              model="gpt-4o",
              response_format={ "type": "json_object" },
              messages=[
                {"role": "system", "content": system_prompt.strip()},
                {"role": "user", "content": user_prompt.strip()}
              ]
            )

            res = json.loads(response.choices[0].message.content.strip())
            res['id'] = card['id']
            eval_results.append(res)
            
        except Exception as e:
            print(ru_finn['id'])
    return cards, eval_results

In [210]:
def improve_complete_cards_and_evaluate(ru_finn_cards, ru_eng_cards, ru_eng_evals, model=GPT_4o):
    cards = []
    eval_results = []
    for ru_finn, ru_eng, eval_result in tqdm(list(zip(ru_finn_cards, ru_eng_cards, ru_eng_evals))):
        try:
            ru_word, ru_sentence, finn_word, finn_sentence, card_id = ru_finn.values()
            _, _, eng_word, eng_sentence, _ = ru_eng.values()

            if ru_sentence == '':
                #cards.append(ru_eng)
                #eval_results.append(eval_result)
                continue
            
            tr_is_correct, tr_explanation, tr_suggested_fix = eval_result['translationAccuracy'].values()
            w_is_correct, w_explanation, w_suggested_fix, w_suggested_fix_s = eval_result['wordUsage'].values()
            s_is_correct, s_explanation, s_suggested_fix = eval_result['englishSentence'].values()

            if all([tr_is_correct, w_is_correct, s_is_correct]):
                #cards.append(ru_eng)
                #eval_results.append(eval_result)
                continue
                
            system_prompt = '''You are a multilingual assistant who is proficient in Russian, Finnish and English.'''

            user_prompt = f"""
            Translate the given Russian word or phrase into English and use it within the provided Russian sentence to create a natural and accurate English translation. The Finnish word or sentence can be used as additional context to refine the translation if needed, but the focus should remain on accurately conveying the meaning of the Russian source. Aim for clarity, naturalness, and contextual appropriateness in English.  

            Details to guide your translation:  

            - **Russian word or phrase:** '{ru_word}'  
            - **Finnish word or phrase:** '{finn_word}'  
            - **Russian sentence:** '{ru_sentence}'  
            - **Finnish sentence:** '{finn_sentence}'  
            - **Existing English translation of the word or phrase:** '{eng_word}'  
            - **Existing English translation of the sentence:** '{eng_sentence}'  
            - **Evaluation results of the existing translation:** {eval_result}  

            ### Instructions:  
            1. Translate the word or phrase into English, ensuring it fits naturally into the sentence while maintaining the intended meaning.  
            2. Translate the Russian sentence into English, incorporating the translated word or phrase naturally and accurately.  
            3. Use synonyms or related terms if necessary to ensure the translation sounds fluent and idiomatic in English.  
            4. Consider the Finnish word or sentence and evaluation results as optional aids; use them only if they help clarify meaning or improve naturalness.  

            The goal is to create a polished and natural translation that prioritizes the Russian source meaning, aligns well with the given sentence, and respects the context provided. Return the English translated word or phrase and the English translated sentence, separated by a semicolon (;).
            """

            response = client.chat.completions.create(
              model=model,
              messages=[
                {"role": "system", "content": system_prompt.strip()},
                {"role": "user", "content": user_prompt.strip()}
              ]
            )
            
            tr_word, tr_sentence = response.choices[0].message.content.strip().split(';')

            card = {}
            card["wordFirstLang"] = ru_word
            card["sentenceFirstLang"] = ru_sentence
            card["wordSecondLang"] = tr_word.strip()
            card["sentenceSecondLang"] = tr_sentence.strip()
            card["id"] = card_id
            cards.append(card)
            
            user_prompt = f"""
            Evaluate the correctness of an English word and sentence based on their translations from Russian and Finnish. You will receive a word in Russian, Finnish, and its translation in English, as well as a sentence in Russian, Finnish, and its translation in English. Your task is to assess the quality of the English sentence, the usage of the English word in the sentence, and the accuracy of the translations from Russian and Finnish to English. For each evaluation point, provide a detailed explanation of your judgment and suggest fixes where applicable, either to the English word, the English sentence, or both.

            Please ensure that the English sentence is grammatically correct and natural. Suggest a corrected version if necessary. Verify that the English sentence contains the English word in some form and suggest using synonyms or related terms if the word is missing. Prioritize naturalness and correctness. Ensure that the translations of both the word and sentence from Russian and Finnish to English are accurate and provide corrections if necessary.

            Respond in JSON format with the following structure:
            {{
              "englishSentence": {{
                "isCorrect": true/false,
                "explanation": "Detailed explanation if there is an issue or why it's correct.",
                "suggestedFix": "Suggested corrected sentence if there is an issue, or null if not applicable."
              }},
              "wordUsage": {{
                "isCorrect": true/false,
                "explanation": "Detailed explanation if there is an issue or why it's correct.",
                "suggestedFixSentence": "Suggested corrected sentence if the word usage is incorrect, or null if not applicable.",
                "suggestedFixWord": "Suggested corrected word if the word usage is incorrect, or null if not applicable."
              }},
              "translationAccuracy": {{
                "isCorrect": true/false,
                "explanation": "Detailed explanation if there is an issue or why it's correct.",
                "suggestedFix": "Suggested correction for translation issues, or null if not applicable."
              }}
            }}

            Here are the provided word and sentence in Russian, Finnish, and English:

            - Word in Russian: {ru_word}
            - Word in Finnish: {finn_word}
            - Word in English: {tr_word}
            - Sentence in Russian: {ru_sentence}
            - Sentence in Finnish: {finn_sentence}
            - Sentence in English: {tr_sentence}

            Please adhere to this structure to ensure clear, actionable feedback for each evaluation point.
            """
            
            response = client.chat.completions.create(
              model="gpt-4o",
              response_format={ "type": "json_object" },
              messages=[
                {"role": "system", "content": system_prompt.strip()},
                {"role": "user", "content": user_prompt.strip()}
              ]
            )

            res = json.loads(response.choices[0].message.content.strip())
            res['id'] = card['id']
            eval_results.append(res)
            
        except Exception as e:
            print(ru_finn['id'])
    return cards, eval_results

In [201]:
tmp_eval = get_eval_results_from_file(file_names[5])
tmp_ru_eng = get_ru_eng_cards_from_file(file_names[5])
tmp_ru_finn = get_ru_finn_cards_from_file(file_names[5])

In [202]:
tmp_cards, tmp_evals = generate_complete_cards_and_evaluate(tmp_ru_finn)

  0%|          | 0/239 [00:00<?, ?it/s]

2237


In [203]:
new_tmp_ru_finn = []
ids = [card['id'] for card in tmp_cards]
for card in tmp_ru_finn:
    if card['id'] in ids:
        new_tmp_ru_finn.append(card)

In [211]:
upd_cards, upd_evals = improve_complete_cards_and_evaluate(new_tmp_ru_finn, tmp_cards, tmp_evals)

  0%|          | 0/185 [00:00<?, ?it/s]

In [197]:
[res for res in upd_evals if not res['translationAccuracy']['isCorrect']]

[{'englishSentence': {'isCorrect': True,
   'explanation': 'The English sentence is grammatically correct and sounds natural.',
   'suggestedFix': None},
  'wordUsage': {'isCorrect': True,
   'explanation': "The word 'district' is correctly used in the English sentence. It aligns well with its counterparts in the Russian ('район') and Finnish ('alue') sentences.",
   'suggestedFixSentence': None,
   'suggestedFixWord': None},
  'translationAccuracy': {'isCorrect': False,
   'explanation': "The word 'district' is a correct translation generally, but given the context of the sentence, it's more natural to use 'area' instead of 'district' in English. The word 'district' can sometimes imply a more formal or administrative subdivision, which may not be necessary here.",
   'suggestedFix': 'The apartment is located in a quiet area.'},
  'id': 2037},
 {'englishSentence': {'isCorrect': True,
   'explanation': "The sentence 'I am extremely hungry.' is grammatically correct and sounds natural in

In [212]:
sum([res['translationAccuracy']['isCorrect'] for res in upd_evals]) / len([res['translationAccuracy']['isCorrect'] for res in upd_evals])

0.7586206896551724

In [213]:
sum([res['englishSentence']['isCorrect'] for res in upd_evals]) / len([res['englishSentence']['isCorrect'] for res in upd_evals])

0.6551724137931034

In [214]:
len(tmp_evals), len(upd_evals)

(185, 29)

In [174]:
new_tmp_ru_finn = []
ids = [card['id'] for card in tmp_cards]
for card in tmp_ru_finn:
    if card['id'] in ids:
        new_tmp_ru_finn.append(card)

In [170]:
sum([res['translationAccuracy']['isCorrect'] for res in tmp_evals]) / len([res['translationAccuracy']['isCorrect'] for res in tmp_evals])

0.9086021505376344

In [166]:
for res in tmp_evals:
    if 'translationAccuracy' not in res.keys():
        print(res)
        break

{'EnglishSentence': {'isCorrect': True, 'explanation': "The sentence 'We traveled abroad.' is grammatically correct and sounds natural in English.", 'suggestedFix': None}, 'WordUsage': {'isCorrect': True, 'explanation': "The word 'abroad' is accurately used in the sentence 'We traveled abroad.' and fits the context perfectly.", 'suggestedFixSentence': None, 'suggestedFixWord': None}, 'TranslationAccuracy': {'isCorrect': True, 'explanation': "The translations from Russian 'Мы поехали за границу.' and Finnish 'Me menimme ulkomaille.' to English 'We traveled abroad.' are accurate.", 'suggestedFix': None}, 'id': 2208}
